<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/Simulation_1_2_Mass_Energy_Information_Equivalence_in_Particle_Decayv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Simulation 1.2: NGFT Mass-Energy-Information Equivalence
# Language: Julia

# --- 1. SETUP ---
using Plots
gr() # Use the default GR backend for plotting

# --- 2. NGFT and Physical Constants ---
# We use "natural" units for simplicity, where constants are 1.0.
# This makes Energy and Information directly comparable (E=I).
const c = 1.0       # Speed of Light
const k = 1.0       # Thermodynamic Ratio Constant
const a_U = 1.0     # Universal Information-Energy Constant

# Helper constant from the NGFT energy equation E = (k+1)a_U * I
const NGFT_FACTOR = (k + 1) * a_U

# --- 3. Particle and NGFT Definitions ---
struct Particle
    mass::Float64
    position::Vector{Float64}
    velocity::Vector{Float64}

    # NGFT Properties
    I_rest::Float64
    I_kin::Float64
    I_total::Float64
    E_ngft::Float64
end

# NGFT Conversion Functions based on the formal proof
function calculate_rest_info(mass)
    # from m*c^2 = NGFT_FACTOR * I_rest
    return (mass * c^2) / NGFT_FACTOR
end

function calculate_kinetic_info(momentum_mag)
    # from p*c = NGFT_FACTOR * I_kin
    return (momentum_mag * c) / NGFT_FACTOR
end

function calculate_total_info(I_rest, I_kin)
    # from Pythagorean Information Relation
    return sqrt(I_rest^2 + I_kin^2)
end

# --- 4. The Decay Physics Function ---
function simulate_decay(parent_mass::Float64, daughter_mass::Float64)
    println("--- Simulating Decay ---")

    # A. Initial State: Parent Particle
    I_rest_parent = calculate_rest_info(parent_mass)
    I_kin_parent = 0.0 # It's at rest
    I_total_parent = calculate_total_info(I_rest_parent, I_kin_parent)
    E_ngft_parent = NGFT_FACTOR * I_total_parent

    parent = Particle(parent_mass, [0.0, 0.0], [0.0, 0.0],
                      I_rest_parent, I_kin_parent, I_total_parent, E_ngft_parent)

    println("Parent Particle:")
    println("  - Mass (m₀): $(parent.mass)")
    println("  - Rest Info (I₀): $(parent.I_rest)")
    println("  - TOTAL NGFT ENERGY (Initial): $(parent.E_ngft)\n")

    # B. Decay Calculation (Standard Relativity)
    if parent_mass <= 2 * daughter_mass
        error("Decay not possible: Parent mass must be greater than total daughter mass.")
    end

    # Energy released as kinetic energy
    energy_released = parent.mass * c^2 - 2 * daughter_mass * c^2
    ke_per_daughter = energy_released / 2

    # Total energy of one daughter particle
    total_energy_daughter = daughter_mass * c^2 + ke_per_daughter

    # Use E^2 = (pc)^2 + (mc^2)^2 to find momentum
    momentum_mag_sq = (total_energy_daughter^2 - (daughter_mass * c^2)^2) / c^2
    momentum_mag = sqrt(momentum_mag_sq)

    # Velocity of daughters (from relativistic momentum p = γmv)
    gamma = total_energy_daughter / (daughter_mass * c^2)
    velocity_mag = c * sqrt(1 - 1/gamma^2)

    # C. Final State: Daughter Particles
    # Daughter 1 flies right, Daughter 2 flies left
    vel_d1 = [velocity_mag, 0.0]
    vel_d2 = [-velocity_mag, 0.0]

    I_rest_daughter = calculate_rest_info(daughter_mass)
    I_kin_daughter = calculate_kinetic_info(momentum_mag)
    I_total_daughter = calculate_total_info(I_rest_daughter, I_kin_daughter)
    E_ngft_daughter = NGFT_FACTOR * I_total_daughter

    daughter1 = Particle(daughter_mass, [0.0, 0.0], vel_d1,
                         I_rest_daughter, I_kin_daughter, I_total_daughter, E_ngft_daughter)
    daughter2 = Particle(daughter_mass, [0.0, 0.0], vel_d2,
                         I_rest_daughter, I_kin_daughter, I_total_daughter, E_ngft_daughter)

    # D. Final Verification
    total_final_ngft_energy = daughter1.E_ngft + daughter2.E_ngft

    println("Daughter Particles (each):")
    println("  - Mass (m₀): $(daughter1.mass)")
    println("  - Momentum (p): $(momentum_mag)")
    println("  - Rest Info (I₀): $(daughter1.I_rest)")
    println("  - Kinetic Info (I_kin): $(daughter1.I_kin)")
    println("  - Total Info (I_total): $(daughter1.I_total)")
    println("  - NGFT Energy (E_ngft): $(daughter1.E_ngft)\n")

    println("--- Verification ---")
    println("Initial NGFT Energy: $(parent.E_ngft)")
    println("Final NGFT Energy (D1 + D2): $(total_final_ngft_energy)")

    conservation_check = isapprox(parent.E_ngft, total_final_ngft_energy)
    println("NGFT Information-Energy Conserved? --> $(conservation_check)\n")

    if !conservation_check
        println("WARNING: Conservation law appears violated. Check calculations.")
    end

    return parent, daughter1, daughter2
end


# --- 5. Run Simulation & Prepare for Animation ---
PARENT_MASS = 100.0
DAUGHTER_MASS = 40.0

parent, d1, d2 = simulate_decay(PARENT_MASS, DAUGHTER_MASS)

# --- 6. Animation ---
println("Preparing animation...")

# A. Create the text for the info panel
info_text = """
NGFT Particle Decay Verification

Initial State (Parent Particle):
  Mass (m₀): $(parent.mass)
  Rest Info (I₀): $(round(parent.I_rest, digits=2))
  Kinetic Info (I_kin): $(round(parent.I_kin, digits=2))
  TOTAL INFO-ENERGY: $(round(parent.E_ngft, digits=2))

Final State (Daughter Particles):
  Mass (each): $(d1.mass)
  Momentum (each): $(round(sqrt(sum(d1.velocity.^2 .* d1.mass^2)), digits=2))
  Rest Info (each): $(round(d1.I_rest, digits=2))
  Kinetic Info (each): $(round(d1.I_kin, digits=2))
  Total Info-Energy (each): $(round(d1.E_ngft, digits=2))

Conservation Check:
  Initial Energy: $(round(parent.E_ngft, digits=2))
  Final Energy (D1+D2): $(round(d1.E_ngft + d2.E_ngft, digits=2))
  RESULT: CONSERVED
"""

# B. Create the animation loop
anim = @animate for i in 1:150
    # Create a 2x1 layout
    p_layout = @layout [a{0.5w} b]
    p = plot(layout=p_layout, size=(1200, 600), background_color=:black, legend=false)

    # Subplot 1: Particle visualization
    plot!(p[1], xlims=(-10, 10), ylims=(-2, 2), title="Particle Decay",
          grid=false, xticks=false, yticks=false, border_color=:white)

    if i < 30 # Before decay
        scatter!(p[1], [parent.position[1]], [parent.position[2]], markersize=20, color=:cyan, markerstrokewidth=0)
    else # After decay
        time_after_decay = (i - 30) * 0.1
        pos_d1 = d1.velocity .* time_after_decay
        pos_d2 = d2.velocity .* time_after_decay

        scatter!(p[1], [pos_d1[1]], [pos_d1[2]], markersize=12, color=:magenta, markerstrokewidth=0)
        scatter!(p[1], [pos_d2[1]], [pos_d2[2]], markersize=12, color=:magenta, markerstrokewidth=0)
    end

    # Subplot 2: Bar chart for verification
    plot!(p[2], title="Information-Energy Conservation", ylims=(0, parent.E_ngft*1.1),
          bg_inside=:transparent)

    categories = ["Initial (Parent)", "Final (Daughters)"]
    energies = [parent.E_ngft, d1.E_ngft + d2.E_ngft]

    bar!(p[2], categories, energies, color=[:cyan, :magenta], legend=false,
         bar_width=0.5, ylabel="Total NGFT Energy")
end

# C. Save the animation as a GIF
gif(anim, "ngft_decay_simulation.gif", fps=30)
println("Animation saved as ngft_decay_simulation.gif")

# D. Also save the info text to a file
open("decay_results.txt", "w") do f
    write(f, info_text)
end
println("Calculation results saved to decay_results.txt")

--- Simulating Decay ---
Parent Particle:
  - Mass (m₀): 100.0
  - Rest Info (I₀): 50.0
  - TOTAL NGFT ENERGY (Initial): 100.0

Daughter Particles (each):
  - Mass (m₀): 40.0
  - Momentum (p): 30.0
  - Rest Info (I₀): 20.0
  - Kinetic Info (I_kin): 15.0
  - Total Info (I_total): 25.0
  - NGFT Energy (E_ngft): 50.0

--- Verification ---
Initial NGFT Energy: 100.0
Final NGFT Energy (D1 + D2): 100.0
NGFT Information-Energy Conserved? --> true

Preparing animation...
Animation saved as ngft_decay_simulation.gif
Calculation results saved to decay_results.txt


[ Info: Saved animation to /content/ngft_decay_simulation.gif
